In [ ]:
import numpy as np
import requests
import distance_mat
import ant_colony_opt
import map_generator

OW_KEY = "9139450c44e00d180bec88e243c971e4"

OW_ENDPOINT = "http://api.openweathermap.org/geo/1.0/direct"


with open('cities.txt',mode='r') as file:
    cities = file.readlines()

for i in range(len(cities)):
    cities[i] = cities[i].replace("\n","")

# co_ordinates = []
# for city in cities:
#     PARAMS = {
#         "q": f"{city},IN",
#         "limit": 1,
#         "appid": OW_KEY,
#     }
#     print(city)
#     response = requests.get(url=OW_ENDPOINT, params=PARAMS).json()[0]
#     co_ordinates.append((response['lat'], response['lon']))
#
# print(co_ordinates)
# print(len(co_ordinates))
co_ordinates = [(25.3356491, 83.0076292), (16.4329976, 80.9937151), (14.6783221, 77.6065039), (17.360589, 78.4740613), (25.4381302, 81.8338005), (9.2844657, 79.3125553), (19.7668121, 74.4754386), (25.1737019, 75.8574194), (26.9154576, 75.8189817), (28.6517178, 77.2219388), (23.0216238, 72.5797068), (22.5414185, 88.35769124388872), (23.2584857, 77.401989), (34.0747444, 74.8204443), (25.6093239, 85.1235252), (23.3700501, 85.3250387), (23.1608938, 79.9497702), (19.0785451, 72.878176), (31.1041526, 77.1709729), (23.7952809, 86.4309638), (25.6618755, 94.1019156), (24.7991162, 93.9364419), (25.5760446, 91.8825282), (14.4493717, 79.9873763), (13.0836939, 80.270186), (28.0159286, 73.3171367), (26.4609135, 80.3217588), (12.9767936, 77.590082), (20.4686, 85.8792), (9.931308, 76.2674136)]

dist_mat = distance_mat.Distance_matrix(co_ordinates)

distance_matrix = dist_mat.create_distance_mat()

# Define parameters
num_cities = len(distance_matrix)
num_episodes = 2000
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1

# Initialize Q-table
Q = np.zeros((num_cities, num_cities))


# Define epsilon-greedy policy
def epsilon_greedy_policy(state, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(range(num_cities))
    else:
        return np.argmax(Q[state, :])


# Run Q-learning algorithm
for episode in range(num_episodes):
    # Start from a random city
    current_city = np.random.choice(range(num_cities))
    total_reward = 0
    visited_cities = [current_city]

    # Perform a full tour
    for _ in range(num_cities - 1):
        next_city = epsilon_greedy_policy(current_city, epsilon)
        while next_city in visited_cities:
            next_city = epsilon_greedy_policy(current_city, epsilon)

        reward = -distance_matrix[current_city, next_city]

        # Update Q-table using Q-learning update rule
        Q[current_city, next_city] += learning_rate * (
                    reward + discount_factor * np.max(Q[next_city, :]) - Q[current_city, next_city])

        total_reward += reward
        visited_cities.append(next_city)
        current_city = next_city

    # Return to the starting city
    reward = -distance_matrix[current_city, visited_cities[0]]
    total_reward += reward
    Q[current_city, visited_cities[0]] += learning_rate * (
                reward + discount_factor * np.max(Q[visited_cities[0], :]) - Q[current_city, visited_cities[0]])

    # Print total reward for each episode
    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

# Initialize variables to track the path and its length
best_ans=10000000
visited_tour=[0]
for x in range(num_cities-1):
    current_city = x
    total_length = 0
    visited_cities = [current_city]

    # Perform a full tour
    for _ in range(num_cities - 1):
        next_city = np.argmax(Q[current_city, :])
        while next_city in visited_cities:
            Q[current_city, next_city] = -100000
            next_city = np.argmax(Q[current_city, :])

        visited_cities.append(next_city)
        total_length += distance_matrix[current_city, next_city]
        current_city = next_city

        # Break out of the loop if all cities have been visited and returned to the starting city
        if len(visited_cities) == num_cities and current_city == 0:
            break
    total_length+=distance_matrix[current_city,x]
    visited_cities.append(x)
    if(best_ans>total_length):
        visited_tour=visited_cities
        best_ans=min(best_ans,total_length)
# Print the optimal tour and its length
print("Optimal Tour:", visited_tour)
print("Total Length of Journey:", best_ans)
map = map_generator.Map_generator(cities,co_ordinates,visited_tour)